#### Q1

The probability of rain on a given calendar day in Vancouver is p[i], where i is the day's index. For example, p[0] is the probability of rain on January 1st, and p[10] is the probability of precipitation on January 11th. Assume the year has 365 days (i.e., p has 365 elements). What is the chance it rains more than n (e.g., 100) days in Vancouver? Write a function that accepts p (probabilities of rain on a given calendar day) and n as input arguments and returns the possibility of raining at least n days.

In [3]:
from typing import Sequence

def prob_rain_more_than_n(p: Sequence[float], n: int) -> float:
    
    dp = [0.0] * (len(p) + 1)
    dp[0] = 1.0
    
    for i in p:
        for k in range(len(p), 0, -1):
            dp[k] = dp[k] * (1 - i) + dp[k - 1] * i
        dp[0] *= (1 - i)
    
    total = sum(dp[n:])
    return total

#### Q2

A phoneme is a sound unit (similar to a character for text). We have an extensive pronunciation dictionary (think millions of words). Below is a snippet: ABACUS AE B AH K AH S BOOK B UH K THEIR DH EH R THERE DH EH R TOMATO T AH M AA T OW TOMATO T AH M EY T OW
Given a sequence of phonemes as input (e.g. ["DH", "EH", "R", "DH", "EH", "R"]), find all the combinations of the words that can produce this sequence (e.g. [["THEIR", "THEIR"], ["THEIR", "THERE"], ["THERE", "THEIR"], ["THERE", "THERE"]]). You can preprocess the dictionary into a different data structure if needed.

In [5]:
from typing import List, Sequence, Dict, Tuple
from collections import defaultdict

def preprocess_dictionary(dictionary: List[Tuple[str, List[str]]]) -> Dict[Tuple[str, ...], List[str]]:
    phoneme_to_words = defaultdict(list)
    for word, phonemes in dictionary:
        phoneme_to_words[tuple(phonemes)].append(word)
    return phoneme_to_words

def find_word_combos_with_pronunciation(
    phonemes: Sequence[str],
    phoneme_to_words: Dict[Tuple[str, ...], List[str]]) -> List[List[str]]:

    def backtrack(index: int) -> List[List[str]]:
        if index == len(phonemes):
            return [[]]  
        
        combos = []
        for length in range(1, len(phonemes) - index + 1):
            phoneme_slice = tuple(phonemes[index:index + length])
            if phoneme_slice in phoneme_to_words:
                for word in phoneme_to_words[phoneme_slice]:
                    for rest in backtrack(index + length):
                        combos.append([word] + rest)
        return combos

    return backtrack(0)

pronunciation_dict = [
    ("ABACUS", ["AE", "B", "AH", "K", "AH", "S"]),
    ("BOOK", ["B", "UH", "K"]),
    ("THEIR", ["DH", "EH", "R"]),
    ("THERE", ["DH", "EH", "R"]),
    ("TOMATO", ["T", "AH", "M", "AA", "T", "OW"]),
    ("TOMATO", ["T", "AH", "M", "EY", "T", "OW"]),
]

phoneme_to_words = preprocess_dictionary(pronunciation_dict)
phoneme_sequence = ["DH", "EH", "R", "DH", "EH", "R"]

result = find_word_combos_with_pronunciation(phoneme_sequence, phoneme_to_words)
print(result)


[['THEIR', 'THEIR'], ['THEIR', 'THERE'], ['THERE', 'THEIR'], ['THERE', 'THERE']]


#### Q3

Implement CTC as this [paper](https://dl.acm.org/doi/abs/10.1145/1143844.1143891m) describes. Your implementation should support both forward and backward propagation operations.

In [8]:
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / np.sum(e_x, axis=1, keepdims=True)

def initialize_alpha_beta(T, S):
    alpha = np.zeros((T, S))
    beta = np.zeros((T, S))
    return alpha, beta

def forward_pass(logits, target, blank=0):
    T, V = logits.shape
    S = len(target)
    target_seq = [blank] + [t for t in target for _ in (0, 1)] + [blank]
    S = len(target_seq)

    alpha, _ = initialize_alpha_beta(T, S)
    alpha[0, 0] = logits[0, blank]
    alpha[0, 1] = logits[0, target_seq[1]]

    for t in range(1, T):
        for s in range(S):
            if s == 0:
                alpha[t, s] = alpha[t - 1, s]
            else:
                alpha[t, s] = alpha[t - 1, s] + alpha[t - 1, s - 1]
                if s > 1 and target_seq[s] != target_seq[s - 2]:
                    alpha[t, s] += alpha[t - 1, s - 2]
            alpha[t, s] *= logits[t, target_seq[s]]

    return alpha, target_seq

def backward_pass(logits, target_seq, blank=0):
    T, V = logits.shape
    S = len(target_seq)

    _, beta = initialize_alpha_beta(T, S)
    beta[-1, -1] = logits[-1, blank]
    beta[-1, -2] = logits[-1, target_seq[-2]]

    for t in range(T - 2, -1, -1):
        for s in range(S - 1, -1, -1):
            if s == S - 1:
                beta[t, s] = beta[t + 1, s]
            else:
                beta[t, s] = beta[t + 1, s] + beta[t + 1, s + 1]
                if s < S - 2 and target_seq[s] != target_seq[s + 2]:
                    beta[t, s] += beta[t + 1, s + 2]
            beta[t, s] *= logits[t, target_seq[s]]

    return beta

def ctc_loss(logits, target, blank=0):
    T, V = logits.shape
    alpha, target_seq = forward_pass(logits, target, blank)
    beta = backward_pass(logits, target_seq, blank)

    Z = alpha[-1, -1] + alpha[-1, -2]
    loss = -np.log(Z)

    grad = np.zeros_like(logits)
    for t in range(T):
        for v in range(V):
            mask = np.array([target_seq[s] == v for s in range(len(target_seq))])
            grad[t, v] = -np.sum((alpha[t, mask] * beta[t, mask]) / logits[t, v]) / Z

    return loss, grad